In [ ]:
import cv2 
import tkinter as tk
from tkinter import filedialog, DoubleVar, HORIZONTAL, Button, Label, Scale
from PIL import Image, ImageTk
import numpy as np

chemin_img = None
img = None
original_image = None
tk_image = None

def Charger_image():
    global chemin_img, img, original_image, tk_image
    chemin_img = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.jpeg *.png *.bmp *.gif")])
    if chemin_img:
        img = cv2.imread(chemin_img)
        original_image = img.copy()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        tk_image = ImageTk.PhotoImage(image=Image.fromarray(img))
        canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)
        # Réinitialiser les valeurs des curseurs
        v_rouge.set(100)
        v_vert.set(100)
        v_bleu.set(100)

def apply_modifications(*args):
    global img, original_image, tk_image,modified_image
    if original_image is not None:
        # Appliquer les modifications sur l'image
        modified_image = original_image.copy()
        r =  v_rouge.get() / 255.0
        g =  v_vert.get() / 255.0
        b = v_bleu.get() / 255.0
        
        # Vérifier si les curseurs sont au maximum
        if r == 1.0 and g == 1.0 and b == 1.0:
            # Si les curseurs sont au maximum, rendre l'image entièrement blanche
            modified_image[:, :] = [255, 255, 255]

        else:
            # Appliquer les poids aux canaux de couleur
            modified_image[:, :, 0] = np.clip(original_image[:, :, 0] * b, 0, 255).astype(np.uint8)
            modified_image[:, :, 1] = np.clip(original_image[:, :, 1] * g, 0, 255).astype(np.uint8)
            modified_image[:, :, 2] = np.clip(original_image[:, :, 2] * r, 0, 255).astype(np.uint8)



 
        # Assurer que les valeurs restent dans la plage 0-255
        modified_image = np.clip(modified_image, 1, 255)
        # Mettre à jour l'image affichée
        img = cv2.cvtColor(modified_image, cv2.COLOR_BGR2RGB)
        tk_image = ImageTk.PhotoImage(image=Image.fromarray(img))
        canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)
def save_image():
    if modified_image is not None:
        filename = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", ".png"), ("JPEG files", ".jpg"), ("All files", ".")])
        if filename:
            cv.imwrite(filename, cv.cvtColor(modified_img, cv.COLOR_RGB2BGR))
            messagebox.showinfo("Save Image", "Image saved successfully!")

fenetre = tk.Tk()
fenetre.title("Modifier les canaux RGB")

# Variables pour les taux de rouge, vert et bleu
v_rouge = DoubleVar()
v_vert = DoubleVar()
v_bleu = DoubleVar()

canvas = tk.Canvas(fenetre, width=400, height=400)
canvas.pack()

bouton_charger = tk.Button(fenetre, text="Charger une image", command=Charger_image)
bouton_charger.pack()

# Curseurs pour les canaux RVB
s_rouge = Scale(fenetre, variable=v_rouge, from_=0, to=255, orient=HORIZONTAL, label="Rouge (%)", command=apply_modifications)
s_rouge.pack()

s_vert = Scale(fenetre, variable=v_vert, from_=0, to=255, orient=HORIZONTAL, label="Vert (%)", command=apply_modifications)
s_vert.pack()

s_bleu = Scale(fenetre, variable=v_bleu, from_=0, to=255, orient=HORIZONTAL, label="Bleu (%)", command=apply_modifications)
s_bleu.pack()

# Appliquer les modifications initiales lors du chargement de l'image
apply_modifications()

bouton_charger = tk.Button(fenetre, text="sauvegarder une image", command=save_image)
bouton_charger.pack()

fenetre.mainloop()

